# Introduction to Working with Large Language Models
Today's goals are:
- Learn the basics of interacting with LLMs
- Explore LLMs that can consume other modalities beyond text, e.g., images
- Evaluate an LLM

## Interacting with LLMs
Since LLMs are language models, we primarily interact with them using language vis a vis "prompts." These interactions typically involve the following steps:
1. **System Prompt** - Provides the LLM with some context for how it should "think" about the task at hand
2. **User Prompt** - Instructs the LLM to carry out a task
3. **Assisstant Response** - The output of the LLM in response to the system and user prompts
4. Further Interactions - Steps 2 and 3 can be repeated to form a "conversation"
The quality of the output from an LLM depends highly on how we prompt it--as we will see later in the lab. 